# Semantic Kernel Process Framework ハンズオン（シンプル版）

Microsoft Semantic Kernel Process Frameworkの5つの主要ワークフローパターンを学習します。

## 学習内容
1. シーケンス（Sequential Pattern）
2. ファンアウト・ファンイン（Fan-out/Fan-in Pattern）
3. 条件分岐（Conditional Branching Pattern）
4. ループ処理（Loop Pattern）
5. 動的並列処理（Dynamic Parallel Pattern）

## セットアップ

Process Frameworkを使用するために必要な依存関係をインストールします。

In [ ]:
# # 必要な依存関係のインストール
# !pip install semantic-kernel==1.23.1 \
#     pydantic==2.10.6 \
#     python-dotenv==1.0.1 \
#     azure-ai-projects==1.0.0b7 \
#     azure-identity==1.21.0 \
#     openai==1.66.3 \
#     DateTime==5.5

In [7]:
# 必要なライブラリのインポート
import asyncio
import time
from typing import Dict, Any, Optional
from datetime import datetime
from enum import Enum

# Semantic Kernel Process Framework
from semantic_kernel import Kernel
# from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion
from semantic_kernel.processes import ProcessBuilder
from semantic_kernel.processes.local_runtime.local_kernel_process import start as start_process
from semantic_kernel.functions import kernel_function
from semantic_kernel.processes import ProcessBuilder
from semantic_kernel.processes.local_runtime.local_kernel_process import start
from semantic_kernel.processes.local_runtime.local_event import KernelProcessEvent
from semantic_kernel.processes.kernel_process import KernelProcessStep, KernelProcessStepContext


# Kernel初期化
kernel = Kernel()
print("Kernel準備完了")

Kernel準備完了


## 1. シーケンス（Sequential Pattern）

最もシンプルなワークフローパターンです。処理が順次実行されます。

フロー: [Start] → [Step A] → [Step B] → [Step C] → [End]

In [14]:
class Events(Enum):
    Start = "Start"
    StartA = "StartA"
    StartB = "StartB"
    StepBCompleted = "StepBCompleted"

class StartStep(KernelProcessStep):
    @kernel_function()
    async def start(self, context: KernelProcessStepContext, user_input: str):
        print("--- START ---")
        await asyncio.sleep(2)
        print(f"User Input: {user_input}")
        await context.emit_event(process_event=Events.StartA, data=user_input)

class AStep(KernelProcessStep):
    @kernel_function()
    async def run_a(self, context: KernelProcessStepContext, data: str):
        print("--- STEP A start ---")
        print(f"Input: {data}")
        await asyncio.sleep(4)
        modified_data_a = f"{data} -> A"
        print(f"Output: {modified_data_a}")
        await context.emit_event(process_event=Events.StartB, data=modified_data_a)

class BStep(KernelProcessStep):
    @kernel_function()
    async def run_b(self, context: KernelProcessStepContext, data: str):
        print("--- STEP B start ---")
        print(f"Input: {data}")
        await asyncio.sleep(6)
        processed_data = f"{data} -> B"
        print(f"Output-B: {processed_data}")
        print("--- END ---")
        await context.emit_event(process_event=Events.StepBCompleted)


In [15]:
async def main(user_input: str):
    # Initialize the process builder
    builder = ProcessBuilder(name="SimpleProcess")

    # Add the steps to the process builder
    start_step = builder.add_step(step_type=StartStep)
    step_a = builder.add_step(step_type=AStep)
    step_b = builder.add_step(step_type=BStep)

    # Define the events and where to send them
    builder.on_input_event(Events.Start).send_event_to(target=start_step, function_name="start", parameter_name="user_input")
    start_step.on_event(Events.StartA).send_event_to(target=step_a, function_name="run_a", parameter_name="data")
    step_a.on_event(Events.StartB).send_event_to(target=step_b, function_name="run_b", parameter_name="data")
    step_b.on_event(Events.StepBCompleted).stop_process()

    # Build the process
    kernel_process = builder.build()

    # Start the process
    await start(
        process=kernel_process,
        kernel=Kernel(),
        initial_event=KernelProcessEvent(id=Events.Start, data=user_input),
    )

In [16]:
await main(user_input="Hello!")

--- START ---
User Input: Hello!
--- STEP A start ---
Input: Hello!
User Input: Hello!
--- STEP A start ---
Input: Hello!
Output: Hello! -> A
--- STEP B start ---
Input: Hello! -> A
Output: Hello! -> A
--- STEP B start ---
Input: Hello! -> A
Output-B: Hello! -> A -> B
--- END ---
Output-B: Hello! -> A -> B
--- END ---


## 2. ファンアウト・ファンイン（Fan-out/Fan-in Pattern）

1つのタスクを複数の並列タスクに分散し、結果をマージします。

フロー: [Start] → [Distributor] → [Task A, Task B, Task C] → [Merger] → [End]

In [20]:
# ファンアウト・ファンインパターンの実装

class FanOutEvents(Enum):
    """ファンアウト・ファンインプロセスのイベント定義"""
    START_DISTRIBUTOR = "start_distributor"
    START_TASK_A = "start_task_a"
    START_TASK_B = "start_task_b"
    START_TASK_C = "start_task_c"
    TASK_A_DONE = "task_a_done"
    TASK_B_DONE = "task_b_done"
    TASK_C_DONE = "task_c_done"
    ALL_TASKS_DONE = "all_tasks_done"

class FanOutDistributor(KernelProcessStep):
    """タスクを複数の並列ステップに分散"""
    
    @kernel_function()
    async def distribute(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanOutDistributor 処理開始")
        
        await asyncio.sleep(1)
        
        print("FanOutDistributor 処理完了")
        
        # 各並列タスクにデータを送信
        await context.emit_event(
            process_event=FanOutEvents.START_TASK_A,
            data=data
        )
        
        await context.emit_event(
            process_event=FanOutEvents.START_TASK_B,
            data=data
        )
        
        await context.emit_event(
            process_event=FanOutEvents.START_TASK_C,
            data=data
        )

class FanOutTaskA(KernelProcessStep):
    """並列タスクA"""
    
    @kernel_function()
    async def process_a(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanOutTaskA 処理開始")
        
        await asyncio.sleep(2)
        
        result = f"{data} -> TaskA"
        
        print("FanOutTaskA 処理完了")
        
        await context.emit_event(
            process_event=FanOutEvents.TASK_A_DONE,
            data=result
        )

class FanOutTaskB(KernelProcessStep):
    """並列タスクB"""
    
    @kernel_function()
    async def process_b(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanOutTaskB 処理開始")
        
        await asyncio.sleep(2)
        
        result = f"{data} -> TaskB"
        
        print("FanOutTaskB 処理完了")
        
        await context.emit_event(
            process_event=FanOutEvents.TASK_B_DONE,
            data=result
        )

class FanOutTaskC(KernelProcessStep):
    """並列タスクC"""
    
    @kernel_function()
    async def process_c(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanOutTaskC 処理開始")
        
        await asyncio.sleep(2)
        
        result = f"{data} -> TaskC"
        
        print("FanOutTaskC 処理完了")
        
        await context.emit_event(
            process_event=FanOutEvents.TASK_C_DONE,
            data=result
        )

class FanInMerger(KernelProcessStep):
    """並列タスクの結果をマージ"""
    
    @kernel_function()
    async def merge_a(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanInMerger TaskAの結果を受信")
        print(f"結果A: {data}")
        await asyncio.sleep(0.5)
    
    @kernel_function()
    async def merge_b(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanInMerger TaskBの結果を受信")
        print(f"結果B: {data}")
        await asyncio.sleep(0.5)
    
    @kernel_function()
    async def merge_c(self, context: KernelProcessStepContext, data: str = "") -> None:
        print("FanInMerger TaskCの結果を受信")
        print(f"結果C: {data}")
        await asyncio.sleep(0.5)
        
        # 最後のタスクの結果を受信したらプロセス完了
        print("FanInMerger すべてのタスク完了")
        
        await context.emit_event(
            process_event=FanOutEvents.ALL_TASKS_DONE,
            data="All tasks completed"
        )

print("ファンアウト・ファンインパターンのクラス定義完了")

ファンアウト・ファンインパターンのクラス定義完了


In [21]:
# ファンアウト・ファンインプロセスの構築と実行

async def build_and_run_fanout_process():
    """ファンアウト・ファンインプロセス構築・実行"""
    
    print("ファンアウト・ファンインプロセスを構築中...")
    
    # ProcessBuilder でプロセスを構築
    process_builder = ProcessBuilder("FanOutFanInProcess")
    
    # ステップを追加
    distributor = process_builder.add_step(FanOutDistributor)
    task_a = process_builder.add_step(FanOutTaskA)
    task_b = process_builder.add_step(FanOutTaskB) 
    task_c = process_builder.add_step(FanOutTaskC)
    merger = process_builder.add_step(FanInMerger)
    
    # イベントの流れを定義
    # 入力 -> Distributor
    process_builder.on_input_event(FanOutEvents.START_DISTRIBUTOR) \
                   .send_event_to(target=distributor, function_name="distribute", parameter_name="data")
    
    # Distributor -> 各並列タスク
    distributor.on_event(FanOutEvents.START_TASK_A) \
              .send_event_to(target=task_a, function_name="process_a", parameter_name="data")
    
    distributor.on_event(FanOutEvents.START_TASK_B) \
              .send_event_to(target=task_b, function_name="process_b", parameter_name="data")
    
    distributor.on_event(FanOutEvents.START_TASK_C) \
              .send_event_to(target=task_c, function_name="process_c", parameter_name="data")
    
    # 各並列タスク -> Merger (各タスクの結果を異なる関数で処理)
    task_a.on_event(FanOutEvents.TASK_A_DONE) \
          .send_event_to(target=merger, function_name="merge_a", parameter_name="data")
    
    task_b.on_event(FanOutEvents.TASK_B_DONE) \
          .send_event_to(target=merger, function_name="merge_b", parameter_name="data")
    
    task_c.on_event(FanOutEvents.TASK_C_DONE) \
          .send_event_to(target=merger, function_name="merge_c", parameter_name="data")
    
    # Merger -> プロセス終了
    merger.on_event(FanOutEvents.ALL_TASKS_DONE) \
          .stop_process()
    
    # プロセスをビルド
    process = process_builder.build()
    
    print("プロセス構築完了")
    print("ファンアウト・ファンインプロセスを実行中...")
    
    # テストケースを実行
    test_cases = ["Hello Parallel", "Fan-out Test"]
    
    for i, test_data in enumerate(test_cases, 1):
        print(f"\nテストケース {i}/{len(test_cases)}")
        print(f"初期入力: '{test_data}'")
        
        start_time = time.time()
        
        try:
            process_context = await start(
                process=process,
                kernel=Kernel(),
                initial_event=KernelProcessEvent(
                    id=FanOutEvents.START_DISTRIBUTOR,
                    data=test_data
                )
            )
            
            # プロセス実行時間を計測
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"実行時間: {execution_time:.2f}秒")
            
            # 少し間隔を空ける
            await asyncio.sleep(0.5)
            
        except Exception as e:
            print(f"エラーが発生しました: {e}")
    
    print("\nすべてのファンアウト・ファンインテストが完了しました")

# ファンアウト・ファンインプロセスの実行
await build_and_run_fanout_process()

ファンアウト・ファンインプロセスを構築中...
プロセス構築完了
ファンアウト・ファンインプロセスを実行中...

テストケース 1/2
初期入力: 'Hello Parallel'
FanOutDistributor 処理開始
FanOutDistributor 処理完了
FanOutTaskA 処理開始
FanOutTaskB 処理開始
FanOutTaskC 処理開始
FanOutDistributor 処理完了
FanOutTaskA 処理開始
FanOutTaskB 処理開始
FanOutTaskC 処理開始
FanOutTaskA 処理完了
FanOutTaskB 処理完了
FanOutTaskC 処理完了
FanInMerger TaskAの結果を受信
結果A: Hello Parallel -> TaskA
FanInMerger TaskBの結果を受信
結果B: Hello Parallel -> TaskB
FanInMerger TaskCの結果を受信
結果C: Hello Parallel -> TaskC
FanOutTaskA 処理完了
FanOutTaskB 処理完了
FanOutTaskC 処理完了
FanInMerger TaskAの結果を受信
結果A: Hello Parallel -> TaskA
FanInMerger TaskBの結果を受信
結果B: Hello Parallel -> TaskB
FanInMerger TaskCの結果を受信
結果C: Hello Parallel -> TaskC
FanInMerger すべてのタスク完了
実行時間: 3.54秒
FanInMerger すべてのタスク完了
実行時間: 3.54秒

テストケース 2/2
初期入力: 'Fan-out Test'
FanOutDistributor 処理開始

テストケース 2/2
初期入力: 'Fan-out Test'
FanOutDistributor 処理開始
FanOutDistributor 処理完了
FanOutTaskA 処理開始
FanOutTaskB 処理開始
FanOutTaskC 処理開始
FanOutDistributor 処理完了
FanOutTaskA 処理開始
FanOutTaskB 処理開始
FanOut

## まとめ

本ハンズオンでは、Semantic Kernel Process Frameworkの基本的なワークフローパターンを学習しました。

### 学習したパターン
1. **シーケンス**: 順次実行パターン
2. **ファンアウト・ファンイン**: 並列処理とマージパターン

### 主要なポイント
- ProcessBuilderによるプロセス構築
- KernelProcessStepによるステップ実装
- KernelProcessEventによるイベント送信
- asyncio.sleep()による処理時間シミュレーション
- シンプルなprint文による処理状況表示

これらのパターンを組み合わせることで、複雑なワークフローを構築できます。